# Using inverted fits image. 

In [1]:
import numpy as np
import dnds_calculations as dnds
from astropy.io import fits

In [2]:
original_fits = 'mosaic_image.fits'
gmrt_catalog = 'mosaic_image.pybdsf.srl.fits'
with fits.open(gmrt_catalog) as hdul:
    data = hdul[1].data
original_fluxes = np.array(data['Total_flux'])

flux_min, flux_max = min(original_fluxes), max(original_fluxes)
print(flux_min)
flux_min = 0.02 if (flux_min<0.02) else flux_min
bins = 11
flux_bins = np.logspace(np.log10(flux_min), np.log10(flux_max), bins)
bin_centers = (flux_bins[:-1] + flux_bins[1:]) / 2

0.010124571214612951


In [3]:
len(original_fluxes)

640

In [4]:
negative_fits = "negative_image.fits"
negative_catalog = negative_fits.replace(".fits", ".pybdsf.srl.fits")

dnds.invert_fits_image(original_fits, negative_fits)

Inverted FITS image saved to negative_image.fits


In [5]:
dnds.run_pybdsf_neg(negative_fits)
# recovered_flux = dnds.read_pybdsf_catalog('negative_image.pybdsf.srl.fits')

stty: 'standard input': Inappropriate ioctl for device
stty: 'standard input': Inappropriate ioctl for device


--> Wrote FITS file 'negative_image.pybdsf.srl.fits'


In [6]:
original_fluxes = dnds.read_pybdsf_catalog(gmrt_catalog)
negative_fluxes = dnds.read_pybdsf_catalog(negative_catalog)

cat_counts, _ = np.histogram(original_fluxes, bins=flux_bins)
neg_counts, _ = np.histogram(np.abs(negative_fluxes), bins=flux_bins)

fdr = np.divide(neg_counts, cat_counts, out=np.zeros_like(neg_counts, dtype=float), where=(cat_counts) > 0)
fdr_correction = np.maximum(1 - fdr, 0.01) # correction factor to be divided with in the code. 
np.save('fdr_correction.npy',fdr_correction)

In [7]:
np.sum(cat_counts)

598

In [8]:
neg_counts, cat_counts

(array([1, 2, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([124, 148, 127, 101,  49,  32,  10,   4,   1,   2]))

In [9]:
fdr_correction

array([0.99193548, 0.98648649, 0.99212598, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [10]:
100*(598-4)/598

99.33110367892976

In [11]:
%

UsageError: Line magic function `%` not found.
